In [198]:
import pandas as pd
import os,re
import numpy as np

In [199]:
parsed = os.listdir('Data')

In [316]:
codes = {
    2011:{'MT':130,'CN':123,'CH':119,'LC':126},
    2012:{'CN':153,'CH':154,'LC':155,'MT':156},
    2013:{'CN':188,'CH':187,'LC':189,'MT':190},
    2015:{'CN':277,'CH':273,'LC':280,'MT':284},
    2016:{'CN':307,'CH':308,'LC':309,'MT':310},
    2017:{'CN':407,'CH':408,'LC':409,'MT':410},
    2018:{'CN':463,'CH':464,'LC':465,'MT':466},
    2019:{'CN':519,'CH':520,'LC':521,'MT':522},
    2020:{'CN':604,'CH':574,'LC':584,'MT':594},
    2021:{'CN':916,'CH':886,'LC':896,'MT':906},
    2022:{'CN':1092,'CH':1062,'LC':1072,'MT':1082}
}

In [326]:
dfs = {}


def match_gab_to_df(df,gab):
    answers=[]
    languages=[]
    used = set()
    for line in df.to_dict(orient='records'):
        if line['test']=='redação':
            answers.append('')
            languages.append('portuguese')
            continue
        possible_gabarito = gab[gab['question']==line['question_number']]
        gabarito = possible_gabarito.iloc[0]
        
        if (str(gabarito['question'])+str(gabarito['test'])) in used:
            gabarito = possible_gabarito.iloc[1]
            
        answers.append(gabarito['answer'])
        languages.append(gabarito['test'] if not pd.isna(gabarito['test']) else 'portuguese')
        used.add(str(gabarito['question'])+str(gabarito['test']))
    return answers,languages


for prova in parsed:
    df = pd.read_csv('Data/'+prova)
    year = int(re.search("\d\d\d\d",prova).group())
    df['question'] = df['question'].apply(lambda question: question.upper())
    df['year'] = year
    df['question_number'] = df['question'].apply(lambda question: int(re.search("[\d]+",question).group()) if re.search("[\d]+",question) else np.nan)
    df['test']='redação'
    if 'D1_' in prova or 'ia1' in prova:
        if year < 2017:
            df.loc[df['question_number']<=45,'test'] = 'CH'
            df.loc[df['question_number']>45,'test'] = 'CN'
        else:
            df.loc[df['question_number']<=45,'test'] = 'LC'
            df.loc[df['question_number']>45,'test'] = 'CH'
    else:
        if year < 2017:
            df.loc[df['question_number']<=135,'test'] = 'LC'
            df.loc[df['question_number']>135,'test'] = 'MT'
        else:
            df.loc[df['question_number']<=135,'test'] = 'CN'
            df.loc[df['question_number']>135,'test'] = 'MT'
    #add gabarito data
    gab = pd.read_csv('GabaritosCsv/'+prova.replace('CAD','GAB')).sort_values(by=['question','test'])
    df['answer'],df['language'] = match_gab_to_df(df,gab)
    #we want to sort dataframe in the following order: english -> spanish -> portuguese then question orders
    #this is useful for the adding of microdata
    df['sort_index'] = df['language'].apply(lambda lang: 0 if lang == 'english' else 1 if lang=='spanish' else 2)
    df = df.sort_values(by=['sort_index','question_number']).drop(columns=['sort_index'])
    
    #add microdata data
    important_columns = ['CO_PROVA','NU_PARAM_A','NU_PARAM_B','NU_PARAM_C','TP_LINGUA','CO_POSICAO','CO_HABILIDADE','TX_GABARITO']
    microdata = pd.read_csv(f'microdados/ITENS_PROVA_{year}.csv',sep=';',usecols=important_columns, encoding='latin-1')
    microdata.loc[microdata['TP_LINGUA'].isna(),'TP_LINGUA']='portuguese'
    microdata.loc[microdata['TP_LINGUA']==0,'TP_LINGUA']='english'
    microdata.loc[microdata['TP_LINGUA']==1,'TP_LINGUA']='spanish'
    
    #divide tests
    for test in df['test'].drop_duplicates().tolist():
        print(prova+' '+ test)
        one_subject_test = df[df['test']==test].copy().reset_index(drop=True)
        if test == 'redação':
            one_subject_test.to_csv(f'ProvasComMicrodados/ENEM_{year}_{test}.csv')
        else:
            CO_PROVA = codes[year][test]
            specific_microdata = microdata[microdata['CO_PROVA']==CO_PROVA].copy()
            one_subject_test['CO_PROVA'] = CO_PROVA
            
            try:
                output = pd.merge(one_subject_test,specific_microdata,left_on=['question_number','language'],right_on=['CO_POSICAO','TP_LINGUA'])
                if not output.empty:
                    raise Exception(f'Failed matching')
            except:
                one_subject_test['match'] = list(one_subject_test.index+1)
                output = pd.merge(one_subject_test,specific_microdata,left_on=['match'],right_on=['CO_POSICAO']).drop(columns=['match'])
                if not (output['TX_GABARITO']!=output['answer']).sum()==0:
                    raise Exception(f'Gabarito and answer should be the same for test {prova} {test}')
                    CO_PROVA = str(CO_PROVA) + 'MAYBEBROKENDONOTUSE'
            if output.empty:
                print('Output empty??')
            output.drop(columns=['CO_PROVA_x','TP_LINGUA','CO_POSICAO'],inplace=True)
            output.rename(columns={'CO_PROVA_y':'CO_PROVA'},inplace=True)
            output['question_number'] = output['question_number'].astype(int)
            output.to_csv(f'ProvasComMicrodados/ENEM_{year}_{test}_CO_PROVA_{CO_PROVA}.csv',index=False,encoding='utf-8')
    dfs[prova] = df

2015_PV_CAD_reaplicacao_PPL_D1_CD11_ledor_branco.csv CH
2015_PV_CAD_reaplicacao_PPL_D1_CD11_ledor_branco.csv CN
2015_PV_CAD_reaplicacao_PPL_D2_CD14_ledor_cinza.csv LC
Output empty??
2015_PV_CAD_reaplicacao_PPL_D2_CD14_ledor_cinza.csv MT
Output empty??
2015_PV_CAD_reaplicacao_PPL_D2_CD14_ledor_cinza.csv redação
2016_PV_CAD_impresso_D1_CD3_ledor_branco.csv CH
Output empty??
2016_PV_CAD_impresso_D1_CD3_ledor_branco.csv CN
Output empty??
2016_PV_CAD_impresso_D2_CD6_ledor_cinza.csv LC
Output empty??
2016_PV_CAD_impresso_D2_CD6_ledor_cinza.csv MT
2016_PV_CAD_impresso_D2_CD6_ledor_cinza.csv redação
2017_PV_CAD_impresso_D1_CD9_laranja.csv LC
2017_PV_CAD_impresso_D1_CD9_laranja.csv CH
Output empty??
2017_PV_CAD_impresso_D1_CD9_laranja.csv redação
2017_PV_CAD_impresso_D2_CD11_laranja.csv CN
Output empty??
2017_PV_CAD_impresso_D2_CD11_laranja.csv MT
Output empty??
2018_PV_CAD_impresso_D1_CD9_laranja.csv LC
2018_PV_CAD_impresso_D1_CD9_laranja.csv CH
2018_PV_CAD_impresso_D1_CD9_laranja.csv redação


Exception: Gabarito and answer should be the same for test ia1_caderno3_CAD_branco_ledor_2011.csv CH

In [327]:
specific_microdata.sort_values(by='CO_POSICAO')

,CO_POSICAO,TX_GABARITO,CO_HABILIDADE,NU_PARAM_A,NU_PARAM_B,NU_PARAM_C,CO_PROVA,TP_LINGUA
809,1,E,21,3.21898,-0.50056,0.36109,119,portuguese
394,2,E,13,1.98100,0.29799,0.14845,119,portuguese
735,3,D,23,0.34715,0.36053,0.02354,119,portuguese
830,4,B,30,2.54487,0.97652,0.12278,119,portuguese
747,5,E,6,2.53827,1.11917,0.18589,119,portuguese
705,6,C,18,2.93542,1.37299,0.13042,119,portuguese
295,7,E,26,1.79484,0.39595,0.44445,119,portuguese
795,8,A,17,1.14241,1.84779,0.17173,119,portuguese
751,9,D,29,2.01408,0.91260,0.43892,119,portuguese
854,10,A,26,1.97592,0.03668,0.24629,119,portuguese


In [328]:
one_subject_test

,question,body,A,B,C,D,E,year,question_number,test,answer,language,CO_PROVA,match
0,QUESTÃO 01,"No mundo árabe, países governados há décadas p...",reforçar a atuação dos regimes políticos exist...,tomar conhecimento dos fatos sem se envolver.,manter o distanciamento necessário à sua segur...,disseminar vírus capazes de destruir programas...,difundir ideias revolucionárias que mobilizara...,2011,1,CH,E,portuguese,119,1
1,QUESTÃO 02,"O Movimento dos Caras-Pintadas, ocorrido no in...",aliou-se aos partidos de oposição e organizou ...,manifestou-se contra a corrupção e pressionou ...,engajou-se nos protestos relâmpago e utilizou ...,espelhou-se no movimento estudantil de 1968 e ...,tornou-se porta-voz da sociedade e influenciou...,2011,2,CH,E,portuguese,119,2
2,QUESTÃO 03,O brasileiro tem noção clara dos comportamento...,"decorrentes da vontade divina e, por esse moti...","parâmetros idealizados, cujo cumprimento é des...",amplas e vão além da capacidade de o indivíduo...,"criadas pelo homem, que concede a si mesmo a l...",cumpridas por aqueles que se dedicam inteirame...,2011,3,CH,D,portuguese,119,3
3,QUESTÃO 04,"A Floresta Amazônica, com toda a sua imensidão...","expansão do Projeto Grande Carajás, com incent...",difusão do cultivo da soja com a implantação d...,"construção da rodovia Transamazônica, com o ob...",criação de áreas extrativistas do látex das se...,ampliação do polo industrial da Zona Franca de...,2011,4,CH,B,portuguese,119,4
4,QUESTÃO 05,O café é tido como uma das maiores riquezas do...,importância sociopolítica das elites cafeicult...,importância dada às tradições na sociedade bra...,admiração despertada pelas elites cafeicultora...,"importância econômica da cafeicultura, uma vez...",utilização da hegemonia política das elites ca...,2011,5,CH,E,portuguese,119,5
5,QUESTÃO 06,O Centro-Oeste apresentou-se como extremamente...,industrialização voltada para o setor de base.,economia da borracha no sul da Amazônia.,fronteira agropecuária que degradou parte do c...,exploração mineral na Chapada dos Guimarães.,extrativismo na região pantaneira.,2011,6,CH,C,portuguese,119,6
6,QUESTÃO 07,Um dos principais objetivos de se dar continui...,a aração.,o terraceamento.,o pousio.,a drenagem.,o desmatamento.,2011,7,CH,E,portuguese,119,7
7,QUESTÃO 08,Uma empresa norte-americana de bioenergia está...,ampliar as regiões produtoras pela adaptação d...,beneficiar os pequenos produtores camponeses d...,abandonar a energia automotiva derivada do pet...,baratear cultivos alimentares substituídos pel...,reduzir o impacto ambiental pela não emissão d...,2011,8,CH,A,portuguese,119,8
8,QUESTÃO 09,A araucária é uma árvore que faz parte de um i...,o avanço do extrativismo de minerais metálicos...,a contínua ocupação agrícola intensiva de grão...,o processo de desmatamento motivado pela expan...,o avanço da indústria de papel e celulose a pa...,o adensamento do processo de favelização sobre...,2011,9,CH,D,portuguese,119,9
9,QUESTÃO 10,"SOBRADINHO O homem chega, já desfaz a natureza...",a migração forçada da população ribeirinha.,o rebaixamento do nível do lençol freático local.,a preservação da memória histórica da região.,a ampliação das áreas de clima árido.,a redução das áreas de agricultura irriga da.,2011,10,CH,A,portuguese,119,10
